In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('model/tmdb_5000_movies.csv')
credits = pd.read_csv('model/tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits, on = 'title')
movies = movies[['id', 'title', 'overview', 'keywords', 'genres', 'cast', 'crew']]
movies.dropna(inplace=True)

In [4]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [5]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [6]:
def convert2(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
            L.append(i['name'])
            count +=1
        else:
            break
    return L

In [7]:
movies['cast'] = movies['cast'].apply(convert2)

In [8]:
def fetch_director(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [9]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [10]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [11]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ', '') for i in x])

In [23]:
movies['tags'] = movies['overview'] + movies['keywords'] + movies['genres'] + movies['cast'] + movies['crew']
movies.to_csv("features.csv",index=False)

In [13]:
tags_df = movies[['id', 'title', 'tags']]

In [14]:
tags_df['tags'] = tags_df['tags'].apply(lambda x: " ".join(x))

<ipython-input-14-67ae15b907bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tags_df['tags'] = tags_df['tags'].apply(lambda x: " ".join(x))


In [15]:
tags_df['tags'] = tags_df['tags'].apply(lambda x: x.lower())

<ipython-input-15-c81f249ff11a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tags_df['tags'] = tags_df['tags'].apply(lambda x: x.lower())


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [17]:
vectors = cv.fit_transform(tags_df['tags']).toarray()

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [21]:
def recommend(movie):
    movie_index = tags_df[tags_df['title'] == movie].index[0]
    distance = similarity[movie_index]
    movie_r = sorted(list(enumerate(distance)), reverse=True, key = lambda x:x[1])[1:6]
    for i in movie_r:
        print(tags_df.iloc[i[0]].title)
recommend('Batman')

Batman
Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Returns
